In [1]:
import pandas as pd
import numpy as np 
import regex as re
from datetime import datetime
from time import time
from time import sleep
from math import ceil
from datetime import timedelta
import unicodedata  
import os
from IPython.display import clear_output

In [2]:
#entering  in the folder to get the data
%cd Data_Source

#listing all the files on the Data_Source folder
files = os.listdir()

#filtering the xlsx or xls format on the directory
lst_excel_files = [x for x in files if x.split('.')[-1]=='xlsx' or  x.split('.')[-1]=='xls']

def get_sheets_names(file):
    """
    This function gets the names of the sheets inside the excel file

    Args: The file which you want extract the name of the sheet inside of it.

    Returns: The sheets names 
    """
    excel = pd.ExcelFile(file)
    return excel.sheet_names  #see all sheet names

#putting all the sheets names inside of a list using a list comprehension that loops through list of excel files(lst_excel files)
sheets_names = [get_sheets_names(excel) for excel in lst_excel_files]

"""
Zipping the lists together for presenting and executing more code, with a list source(lst_source). 
The list source is for unifying sources in a unique list
"""
lst_source = list(zip(lst_excel_files, sheets_names))

#building a data frame with information about the excel files
df_excel= pd.DataFrame(lst_source, columns=['excel_file','sheets'])

#living the diretory of the Data_Source to ensure that the code will be not brake if it executed in the incorrect order.
%cd ..

#looping through the list_source to make a summary of the data that will be ingested on the Pandas DataFrame
for source in lst_source: 
    print(f'EXCEL: {source[0]} >>> SHEETS: {source[1]}')

#Presenting a summary about the excel_files in a DataFrame format
df_excel

/home/roberto/git/projeto_controle_financeiro_gesac-8/Data_Source
/home/roberto/git/projeto_controle_financeiro_gesac-8
EXCEL: Relatorio_SISGESAC 1-02-2022.xls >>> SHEETS: ['Extração']
EXCEL: Proponentes.xlsx >>> SHEETS: ['Sheet1']
EXCEL: Emendas2021_resumo.xlsx >>> SHEETS: ['Planilha1']
EXCEL: Controle de NC 2022.xlsx >>> SHEETS: ['SintéticoNC', 'AnalíticoNC', 'ÍndiceCorreção']
EXCEL: estados.xlsx >>> SHEETS: ['estados']
EXCEL: Solicitação de Financeiro - EPs - Janeiro.xlsx >>> SHEETS: ['Dinâmica', 'RESUMO', 'APTO', 'NÃOAPTO']
EXCEL: Orçamentário MCOM 2021 16-11-21 setel.xlsx >>> SHEETS: ['Adm. Direta - Discricionárias', 'Emendas Individuais', 'Emendas de Bancada', 'dinamica', 'base']
EXCEL: Emendas2021_resumo_ant.xlsx >>> SHEETS: ['Planilha1']


,excel_file,sheets
0,Relatorio_SISGESAC 1-02-2022.xls,[Extração]
1,Proponentes.xlsx,[Sheet1]
2,Emendas2021_resumo.xlsx,[Planilha1]
3,Controle de NC 2022.xlsx,"[SintéticoNC, AnalíticoNC, ÍndiceCorreção]"
4,estados.xlsx,[estados]
5,Solicitação de Financeiro - EPs - Janeiro.xlsx,"[Dinâmica, RESUMO, APTO, NÃOAPTO]"
6,Orçamentário MCOM 2021 16-11-21 setel.xlsx,"[Adm. Direta - Discricionárias, Emendas Indivi..."
7,Emendas2021_resumo_ant.xlsx,[Planilha1]


In [7]:
#entering in the Data_Source directory to get and ingest the data into the DataFrame
%cd Data_Source

"""
Creating a list manually with the information presented about the sheets on the Data_Source Directory.
The First index of these tuples referer to the name that we will give to the DataFrame the second one refers to the sheet that the DataFrame will use.

EXAMPLE:
df_nm =[
(dataframe1_name, sheet_name1),
....
(dataframe8_name, sheet_name8)
]

the df_nm will be used to create the data dictionary inside of a Pandas DataFrame
"""
df_nm =[
('df_an', 'AnalíticoNC'),
# ('df_ei', 'Emendas Individuais'),
('df_sn', 'SintéticoNC'), 
('df_ic', 'ÍndiceCorreção'),
('df_pr', 'Sheet1'),
('df_em', 'Planilha1'),
('df_estados', 'estados'),
('df_sg', 'Extração')
] 

#df_sheets -> This DataFrame was created do organize da data dictionary, it also will useful to some operation through the code
df_sheet = pd.DataFrame(df_nm, columns = ['df_name', 'sheet_name'])

#Creating a column to put the information that indicate from which excel file the DataFrames and sheets come from.
df_sheet['excel_file']= ''

"""Creating a loop to fill the rows in the df_sheets DataFrame with information about the excel source """

for i1, sheets in enumerate(df_excel['sheets']):
    for i2, sheet in enumerate(df_sheet['sheet_name']):
        if sheet in sheets:
            df_sheet['excel_file'][i2] = df_excel['excel_file'][i1]

#living the diretory of the Data_Source to ensure that the code will be not brake if it executed in the incorrect order.
%cd ..

#presenting the information 
df_sheet

[Errno 2] No such file or directory: 'Data_Source'
/home/roberto/git/projeto_controle_financeiro_gesac-8/Data_Source
/home/roberto/git/projeto_controle_financeiro_gesac-8


,df_name,sheet_name,excel_file
0,df_an,AnalíticoNC,Controle de NC 2022.xlsx
1,df_sn,SintéticoNC,Controle de NC 2022.xlsx
2,df_ic,ÍndiceCorreção,Controle de NC 2022.xlsx
3,df_pr,Sheet1,Proponentes.xlsx
4,df_em,Planilha1,Emendas2021_resumo_ant.xlsx
5,df_estados,estados,estados.xlsx
6,df_sg,Extração,Relatorio_SISGESAC 1-02-2022.xls


In [8]:
#entering in the Data_Source directory to get and ingest the data into the DataFrame
%cd Data_Source

def parameters(df_input):
    """
    This function helps to create new DataFrames using the data dictionary in the df_sheets

    ARGS: 
        df_input(str): The name that you wanna get data from.
    RETURNS:
        excel(str): The excel file name where the Dataframe belongs
        sheet(str): The sheet's name that will be used to create the DataFrame
    """
    excel = df_sheet.loc[df_sheet.df_name == df_input]['excel_file'].values[0]
    sheet = df_sheet.loc[df_sheet.df_name == df_input]['sheet_name'].values[0]
    return excel, sheet

excel, sheet = parameters('df_an')
df_an = pd.read_excel(excel, sheet_name='AnalíticoNC',
usecols="B:J", skiprows=1,parse_dates=True)

excel, sheet = parameters('df_sn')
df_sn = pd.read_excel(excel, sheet_name=sheet,
usecols="B:J", skiprows=3,parse_dates=True)

excel, sheet = parameters('df_ic')
df_ic = pd.read_excel(excel, sheet_name=sheet,
skiprows=1,usecols="B:E")

excel, sheet = parameters('df_pr')
df_pr = pd.read_excel(excel)

# excel, sheet = parameters('df_cg')
# df_cg = pd.read_excel(excel, sheet_name=sheet
# , skiprows=12,usecols="c:w")

# excel, sheet = parameters('df_ei')
# df_ei = pd.read_excel(excel)

excel, sheet = parameters('df_em')
df_em= pd.read_excel(excel, sheet_name=sheet)


excel, sheet = parameters('df_estados')
df_estados = pd.read_excel(excel)

excel, sheet = parameters('df_sg')
df_sg= pd.read_excel(excel)


"""
Run loop only once or rerun all!!!
In general, the excel files with just one sheet don't have a good name, so the name was changed to be similar to the name of the excel file
"""

for i, sheet in enumerate(df_excel['sheets']):
    if len(sheet)==1:
        new_name = df_excel['excel_file'][i]
        re_pattern = '(.xlsx?|\d|-|\s)'
        new_name = re.sub(re_pattern,'',new_name)
        df_sheet.sheet_name.loc[df_sheet.sheet_name == sheet[0]] = new_name
    else: pass



for sheet in df_excel['sheets']:
    for index_1, xlsx_1 in enumerate(df_excel['excel_file']):
        for index_2, xlsx_2 in enumerate(df_sheet['excel_file']):
            if xlsx_1 == xlsx_2 and len(df_excel['sheets'][index_1])==1:
                df_excel['sheets'][index_1] = df_sheet['sheet_name'][index_2]
            else: pass

df_pr = df_pr[['prop_pk1','prop_1','prop_2','prop_3','prop_4','prop_pk2', 'uf','partido','orgao']]
#----------------------------------------------------------
#Getting out of the Data_Source folder after having read the files

%cd ..

df_excel

/home/roberto/git/projeto_controle_financeiro_gesac-8/Data_Source


/home/roberto/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/roberto/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


/home/roberto/git/projeto_controle_financeiro_gesac-8


,excel_file,sheets
0,Relatorio_SISGESAC 1-02-2022.xls,Relatorio_SISGESAC
1,Proponentes.xlsx,Proponentes
2,Emendas2021_resumo.xlsx,[Planilha1]
3,Controle de NC 2022.xlsx,"[SintéticoNC, AnalíticoNC, ÍndiceCorreção]"
4,estados.xlsx,estados
5,Solicitação de Financeiro - EPs - Janeiro.xlsx,"[Dinâmica, RESUMO, APTO, NÃOAPTO]"
6,Orçamentário MCOM 2021 16-11-21 setel.xlsx,"[Adm. Direta - Discricionárias, Emendas Indivi..."
7,Emendas2021_resumo_ant.xlsx,Emendas_resumo


In [9]:
def strip_accents(string_input):
    """
    This function remove the accent of string so it can be more ease to make comparison 
    between between different strings, useful information

    ARGS: 
        string(str): The string that you wanna transform, it can be a value in a row or head column.
    RETURNS:
        string_cleaned(str): The sheet's name that will be used to create the DataFrame
    """
    string_input = str(string_input)
    string_cleaned = ''.join(c for c in unicodedata.normalize('NFD', string_input)if unicodedata.category(c) != 'Mn')
    return string_cleaned

def transform_column_name(column_name):
    """
    This function removes the accent on the string so it can be easier to make comparisons
     between different values of the same type,  and find useful information.

    ARGS: 
        column_name(str): the column_name
    RETURNS:
        name_cleaned(str): The column_name after transformed.
    """
    name_cleaned = strip_accents(str(column_name).strip('[]()').lower().replace(' ','_'))
    return name_cleaned

def rename_cl_df(df):
    """
    This function gets the DataFrame and renames its columns using the transform_column_name()

    ARGS:
        df(pandas.core.frame.DataFrame): The DataFrame that you will rename the columns
    RETURN:
        df(pandas.core.frame.DataFrame): The same DataFrame that you entered in the function but transformed.
    """   

    #This  list comprehension gets the name of the DataFrame and make a list
    old_cln = [cln for cln in df.columns.values]

    #This  list comprehension gets the name of the DataFrame and make a list with theses names transformed by the transform_column_name()
    new_cln = [transform_column_name(cln) for cln in df.columns.values]

    #This dictionary(dct) is made with the lists:old_cln and new_cln. So that the columns in the DataFrame can be renamed comparing key and value inside ofthe dictionary.   
    dct = dict(zip(old_cln, new_cln))

    #
    df.rename(columns=dct,inplace=True)
    lst_df_clns.append(new_cln)
    return df
 
 
df_sheet['columns'] = '' 

#creating a list with all dfs from the data dictionary
dfs = [ x for x in df_sheet['df_name']]


cmd = str(dfs).replace("'","") 
exec('dfs = {}'.format(cmd))

lst_df_clns = []
for df in dfs:
    rename_cl_df(df)


df_sheet['columns'] = lst_df_clns 

In [15]:
type(df_sheet)

pandas.core.frame.DataFrame

In [10]:
#AQUI
# criando função 'regexing' p/ subst. nomes abreviados de forma a normalizar as strings
lst_re = [
('PRF ','POLICIA RODOVIARIA FEDERAL '),
('MCOM ', 'MINISTERIO DAS COMUNICACOES '),
('CABO',''),
('BANCADA ','BANCADA-'),
(r'MIN/.',''),
(r'DEP/.|DEP.\s?FEDERAL|DEPUTAD[OA]\s?(FEDERAL)?',''),
(r'(SEN/.|SENADOR|SENADORA)',''),
(r'-D[EOA]\s','-')
]

lamb_str = lambda x:strip_accents(x).upper()
lamb_re = lambda x:regexing(x)
lamb_strip = lambda x: str(x).strip()

df_estados['nome'] = df_estados['nome'].apply(lamb_str)
uf_re = zip(df_estados['nome'], df_estados['sigla'])

for x in list(uf_re):
    lst_re.append(x)

In [ ]:
#criando lista dos DataFrames pelo qual o loop passara referente aos proponentes
dfs = (df_sn, df_an,df_cg, df_em, df_sg)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''

def regexing(value):
   for string in lst_re:
        value = re.sub(string[0],string[1],value)
   return value

pk2 = []
for index, prop in enumerate(df_pr['prop_pk1']):
    if df_pr['orgao'][index] == "senado":
        pk2.append('SEN. ' + prop)
    elif  df_pr['orgao'][index] == "camara":
        pk2.append('DEP. ' + prop)
    else: 
        pk2.append(prop)
df_pr['prop_pk2'] = pk2

#criando lista dos DataFrames pelo qual o loop passara referente aos proponentes
dfs = (df_sn, df_an,df_cg, df_em, df_sg)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''

In [ ]:
df_sg['proponente'] = df_sg['nosolicitante'] 
df_em['proponente'] = df_em['emenda']

lst_replace = [
       ('DEPA.',''),
       ('DEP.',''),
       ('SEN.',''),
       ('BANCADA ','BANCADA-')]

def replace_lst(value):
   value = str(value)
   for string in lst_replace:
        value = re.sub(string[0],string[1],value)
   return value

df_em['proponente'] = df_em.proponente.apply(lambda x: str(x).upper().split('-')[0].split('(')[0])
df_em['proponente'] = df_em.proponente.apply(lambda x: replace_lst(x))
df_em.proponente

In [ ]:

#cleaning analitic sintetic rows
lst = ['SUBTOTAL','TOTAL','NAN','0']

for value in lst:
    df_em = df_em.loc[df_em.proponente != value ]  
    df_sn = df_sn.loc[df_sn.proponente != value ]  
    df_sg = df_sg.loc[df_sg.proponente != value ]  
    
df_sn.fillna(0,inplace=True)
df_an.fillna(0,inplace=True)

#fazendo o loop sobre a lista acima para aplicar as funções que normalizam a coluna com as chaves dos proponentes
lst_df = [df_sn, df_an, df_cg, df_em, df_sg]

for df in lst_df: 
    df['proponente'] = df['proponente'].apply(lamb_str)
    df['proponente'] = df['proponente'].apply(lamb_re)
    df['proponente'] = df['proponente'].apply(lamb_strip)


lst_pr = (df_pr['prop_pk1'], df_pr['prop_1'],df_pr['prop_2'], df_pr['prop_3'],df_pr['prop_4'])

#criando lista das colunas referente ao nome dos proponentes 
def filter_key(x):
    return (x==lst_pr[0])|(x==lst_pr[1])|(x==lst_pr[2])|(x==lst_pr[3])|(x==lst_pr[4])

for df in lst_df:
    lst_prop_pk1 = []
    lst_prop_pk2 = []
    for index,value in enumerate(df.proponente):
        filt_finding_key =  filter_key(df.proponente.iloc[index])
        result1 = str(df_pr.loc[filt_finding_key].prop_pk1.values).strip("[']")
        result2 = str(df_pr.loc[filt_finding_key].prop_pk2.values).strip("[']")
        lst_prop_pk1.append(result1)
        lst_prop_pk2.append(result2)
    df['prop_pk1'] = lst_prop_pk1
    df['prop_pk2'] = lst_prop_pk2

In [ ]:
dfs  = [[df_an,'df_an'], [df_sn,'df_sn'],[df_em,'df_em'], [df_sg,'df_sg']]; 

lst_prop_nk= []; lst = []

for df in dfs:
    for index_2, value in enumerate(df[0].loc[df[0].prop_pk1==''].proponente.values):
        if value not in lst:
            prop_name =  df[0].loc[df[0].prop_pk1==''].proponente.values[index_2]
            prop_source = df[1]
            prop_index = (index_2)
            lst_prop_nk.append((prop_name, prop_source, prop_index))
            lst.append(value)
        else:pass

print('DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:')
df_nk = pd.DataFrame()
df_nk['proponente'] = [x for x,y,z in lst_prop_nk]
df_nk['source'] = [y for x,y,z in lst_prop_nk]

df_nk = df_nk.loc[df_nk.proponente!='FREDERICO'].reset_index(drop=True)

print('ROWS:', df_nk.shape[0])
df_nk

In [ ]:
i = 5

In [ ]:
print(f'PROP: {df_nk.proponente.iloc[i]}.')

def writing_in_df_prop():
    prop_name = df_nk['proponente'][i]
    input_regex = input('REGEX: ')
    name_regex = r''+input_regex
    return prop_name, name_regex

prop_name, name_regex = writing_in_df_prop()
#------------------------------------
filter_cl = ['prop_pk1','prop_1','prop_2','prop_3','prop_4','prop_pk2']
filter_name = df_pr['prop_pk1'].str.match(name_regex) == True; 

df_pr.loc[filter_name][filter_cl]

In [ ]:
write_in_df_pr = True
if write_in_df_pr == True:
    n = input('Type the number of the prop: ')
    df_pr.loc[filter_name,f'prop_{n}']= prop_name
    i+=1

df_pr.loc[filter_name]

In [ ]:
save = False
if save ==True:
    %cd Data_Source
    df_pr.to_excel('Proponentes.xlsx')
    %cd ..

%cd Data_Source
df_pr = pd.read_excel('Proponentes.xlsx')
%cd ..

df_pr.loc[filter_name]


In [ ]:
dfs = [df_an, df_ic]

for index,df in enumerate(dfs): 
    meses = [mes for mes in df.mes.unique()]
    n_mes = ['01/2021','02/2021','03/2021','04/2021','05/2021','06/2021','07/2021','08/2021','09/2021','10/2021','11/2021','12/2021']
    meses = list(zip(meses, n_mes))

    for index,value in enumerate(meses):
        df['mes'].replace(meses[index][0],meses[index][1],inplace=True)


    df['mes']= pd.to_datetime(df['mes'])
    if (df.mes.dtype == '<M8[ns]') == True:
        print(f'The loop is ok!')
        
    else: print('The loops is broke!')
    

In [ ]:
#df_folders
lst_fd = [
    ('FINANCEIRO','\GESAC\GESAC_2017\ADMPP'),
    ('SISGESAC',None)
    ]

fd = [fd for (pj,fd) in lst_fd]
pj = [pj for (pj,fd) in lst_fd]
df_folders = pd.DataFrame({'project':pj,'folder':fd})

In [ ]:
%rm -r PowerBI_Dataset
%mkdir PowerBI_Dataset
%mkdir PowerBI_Dataset/csv/
%mkdir PowerBI_Dataset/xlsx/

%rm -r Data_Dictionary
%mkdir Data_Dictionary
%mkdir Data_Dictionary/csv/
%mkdir Data_Dictionary/xlsx/
%mkdir Data_Dictionary/csv/Columns/
%mkdir Data_Dictionary/xlsx/Columns/

path = 'PowerBI_Dataset'
for i, df  in enumerate(df_sheets['df_name']):
       exec(f'df = {df}')
       exec(f"{df_sheets['df_name'][i]}.to_csv('{path}/csv/{df_sheets['sheet_name'][i]}.csv')")
       exec(f"{df_sheets['df_name'][i]}.to_excel('{path}/xlsx/{df_sheets['sheet_name'][i]}.xlsx')")

path = 'Data_Dictionary'
for df in ('df_sheets', 'df_xlsx', 'df_folders'):
    exec(f"{df}.to_csv('{path}/csv/{df}.csv')")
    exec(f"{df}.to_excel('{path}/xlsx/{df}.xlsx')")

path2 = 'Columns'
for i, df  in enumerate(df_sheets['df_name']):
       exec(f'df = {df}')
       lst_cl = []
       for cl in df.columns: lst_cl.append(cl)
       dct = {'columns':lst_cl}
       exec(f"{df_sheets['df_name'][i]}_columns = pd.DataFrame(dct)")
       exec(f"{df_sheets['df_name'][i]}_columns['descricao'] = '' ")
       exec(f"{df_sheets['df_name'][i]}_columns.to_csv('{path}/csv/{path2}/{df_sheets['sheet_name'][i]}_columns.csv')")
       exec(f"{df_sheets['df_name'][i]}_columns.to_excel('{path}/xlsx/{path2}/{df_sheets['sheet_name'][i]}_columns.xlsx')")